In [1]:
# import the necessary packages
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# CIFAR_10 is a set of 60K images 32x32 pixels on 3 channels
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32

#constant
BATCH_SIZE = 128
NB_EPOCH = 20
NB_CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = RMSprop()

#load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
# convert to categorical
Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(y_test, NB_CLASSES)

# float and normalization
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [4]:
# network - Conv -> RELU -> POOL
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [5]:
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               4194816   
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
__________

In [6]:
# train
model.compile(loss='categorical_crossentropy', optimizer=OPTIM,
metrics=['accuracy'])
model.fit(X_train, Y_train, 
          batch_size=BATCH_SIZE,
          epochs=NB_EPOCH, 
          validation_split=VALIDATION_SPLIT,
          verbose=VERBOSE)
score = model.evaluate(X_test, Y_test,
batch_size=BATCH_SIZE, verbose=VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 74s - loss: 1.7389 - acc: 0.3841 - val_loss: 1.4990 - val_acc: 0.4781
Epoch 2/20
40000/40000 [==============================] - 75s - loss: 1.3816 - acc: 0.5091 - val_loss: 1.4542 - val_acc: 0.4966
Epoch 3/20
40000/40000 [==============================] - 75s - loss: 1.2483 - acc: 0.5575 - val_loss: 1.2043 - val_acc: 0.5827
Epoch 4/20
40000/40000 [==============================] - 74s - loss: 1.1601 - acc: 0.5879 - val_loss: 1.1172 - val_acc: 0.6104
Epoch 5/20
40000/40000 [==============================] - 73s - loss: 1.0832 - acc: 0.6202 - val_loss: 1.0914 - val_acc: 0.6201
Epoch 6/20
40000/40000 [==============================] - 73s - loss: 1.0249 - acc: 0.6379 - val_loss: 1.0673 - val_acc: 0.6371
Epoch 7/20
40000/40000 [==============================] - 71s - loss: 0.9704 - acc: 0.6617 - val_loss: 1.0358 - val_acc: 0.6440
Epoch 8/20
40000/40000 [==============================

In [10]:
#save model
model_json = model.to_json()
open('cifar10_architecture.json', 'w').write(model_json)
model.save('cifar10_weights.h5', overwrite=True)